### Étapes de la fonction

1. 
    -obtenir la liste des joueurs actifs de l'alignement

2.
    -pour chacun d'eux, faire une requête pour savoir combien de points le joueur a eu entre la date de début de      l'alignement et maintenant, en filtrant pour la saison en cours

    -itération en python sur chaque joueur, et un autre df = pd.read_sql juste pour sortir le score

3.    
    -stocker le résultat de la requête 2 dans le champs pointsActuel de la table Alignement (execSQL avec un   UPDATE de la table Alignement)     

In [1]:
##Étape 1
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
requete = "SELECT idAlignement, idPooler, JOUEURS.idNHL, idHockeyRef, prenomJoueur, nomJoueur, position, statutJoueur, dateDebut, dateFin, pointsActuels, salaireActuel FROM ALIGNEMENTS INNER JOIN JOUEURS ON JOUEURS.idNHL = ALIGNEMENTS.idNHL WHERE statutJoueur='Alignement'"
dfJoueursActifs = pd.read_sql(requete, conn)
listIdJoueursActifs = dfJoueursActifs["idNHL"]
dfJoueursActifs

,idAlignement,idPooler,idNHL,idHockeyRef,prenomJoueur,nomJoueur,position,statutJoueur,dateDebut,dateFin,pointsActuels,salaireActuel
0,1,gab,8471675,crosbsi01,Sidney,Crosby,A,Alignement,2020-11-26,2020-12-07,0,8700000
1,2,gab,8478445,barzama01,Matt,Barzal,A,Alignement,2020-10-28,2020-11-14,0,7000000
2,3,hub,8477492,mackina01,Nathan,Mackinnon,A,Alignement,2020-09-28,2020-11-23,0,6300000
3,5,hub,8479394,hartca01,Carter,Hart,G,Alignement,2019-11-25,2020-01-04,0,790000
4,6,cle,8478403,eicheja01,Jack,Eichel,A,Alignement,2019-09-25,2019-11-20,0,10000000
5,8,cle,8479496,rittida01,David,Rittich,G,Alignement,2020-10-30,2020-11-24,0,3000000
6,9,hub,8478445,barzama01,Matt,Barzal,A,Alignement,2020-05-20,2020-10-27,0,7000000


``` SQL
UPDATE ALIGNEMENTS
SET pointsActuels = (SELECT COUNT(Event) AS NbPTS     
  FROM PBP
       INNER JOIN
       ALIGNEMENTS ON (PBP.p1_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p2_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p3_ID = ALIGNEMENTS.idNHL) 
 WHERE EVENT = "GOAL" AND 
       statutJoueur = "Alignement" AND 
       PBP.Date > ALIGNEMENTS.dateDebut AND
       PBP.Date <= ALIGNEMENTS.dateFin
GROUP BY idAlignement)
 ```

In [6]:
majPointsAlignements()

In [ ]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

def majPointsAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pointsActuels = (SELECT NbPTS FROM ((SELECT idAlignement, COUNT(Event) AS NbPTS    FROM PBP INNER JOIN ALIGNEMENTS ON (PBP.p1_ID = ALIGNEMENTS.idNHL) OR (PBP.p2_ID = ALIGNEMENTS.idNHL) OR (PBP.p3_ID = ALIGNEMENTS.idNHL) WHERE EVENT = 'GOAL' AND PBP.Date > ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date > '2019-08-25' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    execSQL(sqlUpdate, conn, write = True)

Une cellule qui:

    - (DONE) (majPoints) qui met à jour les buts, passes et PJ des joueurs selon la période donnée dans Alignements
    - qui met à jour le nombre de points de chaque pooler
    - qui met à jour la masse salariale de chaque pooler
    - (majJson) qui met à jour les json
    - exécute le sync avec GitHub

In [2]:
#conn
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

#execSQL
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

#mettre à jour les buts
def majButsAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET butsActuels = (SELECT NbBUTS FROM ((SELECT idAlignement, COUNT(Event) AS NbBUTS    FROM PBP INNER JOIN ALIGNEMENTS ON (PBP.p1_ID = ALIGNEMENTS.idNHL) WHERE EVENT = 'GOAL' AND PBP.Date > ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date > '2019-08-25' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    execSQL(sqlUpdate, conn, write = True)

#mettre à jour les assist
def majAssistAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET assistActuels = (SELECT NbASSIST FROM ((SELECT idAlignement, COUNT(Event) AS NbASSIST    FROM PBP INNER JOIN ALIGNEMENTS ON (PBP.p2_ID = ALIGNEMENTS.idNHL) OR (PBP.p3_ID = ALIGNEMENTS.idNHL) WHERE EVENT = 'GOAL' AND PBP.Date > ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date > '2019-08-25' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    execSQL(sqlUpdate, conn, write = True)

#mettre à jour les points
def majPointsAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pointsActuels = (butsActuels + assistActuels)"
    execSQL(sqlUpdate, conn, write = True)